In [161]:
import import_ipynb

In [162]:
import rnn_from_scratch as scratch
import stock_price_predictor as spp

In [163]:
def make_sequences(data, sequence_length=50, prediction_period=1):
    x, y = [], []
    for i in range((len(data) - sequence_length - prediction_period)):
        x.append(data[i:i+sequence_length])
        y.append(data[i+sequence_length: i+sequence_length+prediction_period])
    return x, y

In [164]:
def split_sequences(x, y, test_size=0.2):
    test_data_cut = int(test_size * len(x))
    X_train = x[:-test_data_cut]
    y_train = y[:-test_data_cut]
    
    X_test = x[-test_data_cut:]
    y_test = y[-test_data_cut:]
    
    return np.asarray(X_train), np.asarray(y_train), np.asarray(X_test), np.asarray(y_test)

In [165]:
def reshape(data):
    new = []
    for row in data:
        new_row = np.reshape(row, (5,1))
        new.append(new_row)
    return new
        

In [166]:
def prepare_data(data):
    
    # Clean 
    data = spp.clean_data(data)
    data = spp.normalize_data(data, ['Open', 'High', 'Low', 'Close', 'Volume'])
    
    # Reshape 
    data = data.to_numpy()
    data = reshape(data)
    X, y = make_sequences(data)
    X_train, y_train, X_test, y_test = split_sequences(X, y)
    
    return X_train, y_train, X_test, y_test

In [170]:
class LSTM():
    def __init__(self, n_features, hidden_size=50):
        self.n_features = n_features
        self.hidden_size = hidden_size
        self.z_size = hidden_size + vocab_size
        self.parameters = []
        self.training_loss = []
        self.validation_loss = []
        
    def initialize(self):
        self.parameters = scratch.init_lstm(self.hidden_size, self.n_features)
        
    def train(self, train_set, epochs=20, activation='linear'):
        self.training_loss, self.parameters = scratch.train_lstm(train_set, self.parameters, self.hidden_size, epochs, activation)
        
    def validate(self, val_set, epochs):
        self.validation_loss = scratch.validate_lstm(val_set, self.parameters, self.hidden_size, epochs, activation)
        
    def test(self, test_set, idx_to_word=None):
        results = scratch.test_lstm(test_set, self.parameters, self.hidden_size, idx_to_word, activation)
        return results

In [171]:
def main():
    data = spp.get_ticker_data('GOOGL', 10)
    X_train, y_train, X_test, y_test = prepare_data(data)
    
    lstm = LSTM(X_train.shape[-2])
    lstm.initialize()
    lstm.train((X_train, y_train))
#     results = lstm.test(test_set)
#     return results

In [173]:
# main()

In [144]:
X_train.shape

(1973, 50, 5, 1)

In [145]:
X_train[0][0]

array([[0.00914726],
       [0.00898152],
       [0.00991937],
       [0.00979092],
       [0.1296764 ]])

In [135]:
test = X_train[0][0]
new = []
for val in test:
    new.append([val])

new

[[0.00914725785970507],
 [0.008981518570271333],
 [0.009919372041676599],
 [0.009790921473727881],
 [0.1296764020633496]]

In [83]:
# Attention

In [36]:
from pandas import read_csv
import numpy as np
from keras import Model
from keras.layers import Layer
import keras.backend as K
from keras.layers import Input, Dense, SimpleRNN
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.metrics import mean_squared_error

In [37]:
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def show(self):
        print(self.W)
        print(self.b)

In [38]:
test = attention()

In [40]:
test.build([50, 4])

In [41]:
test.show()

<tf.Variable 'att_weight:0' shape=(4, 1) dtype=float32, numpy=
array([[ 0.09407338],
       [ 0.04555698],
       [ 0.0531346 ],
       [-0.04047551]], dtype=float32)>
<tf.Variable 'att_bias:0' shape=(4, 1) dtype=float32, numpy=
array([[0.],
       [0.],
       [0.],
       [0.]], dtype=float32)>
